# Chapter 5: Fractional Differentiation (Concept Notebook)
AFML focus: preserving memory while reducing non-stationarity.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
fd = fracdiff_ffd(uso, d=0.4, thresh=1e-5)
valid = [(i, v) for i, v in enumerate(fd) if not math.isnan(v)]
ix = [i for i, _ in valid]
fv = [v for _, v in valid]

plt.figure(figsize=(10,4))
plt.plot(uso[-300:], label='raw close')
plt.plot(ix[-300:], fv[-300:], label='fracdiff ffd (d=0.4)')
plt.title('Chapter 5: Fractional Differentiation')
plt.legend()
plt.tight_layout()
plt.show()

raw_corr = lag_corr(uso_ret, uso_ret, 1)
fd_ret = simple_returns([v for v in fv])
fd_corr = lag_corr(fd_ret, fd_ret, 1)
print('lag1 autocorr raw returns', raw_corr)
print('lag1 autocorr fracdiff returns', fd_corr)

_ = openquant.filters.cusum_filter_indices(uso, 0.001)


lag1 autocorr raw returns 0.03864416161316631
lag1 autocorr fracdiff returns nan


## Interpretation
Fractional differencing reduces trend-dominated behavior while retaining informative dependence structure.
(Implementation shown as a research helper; OpenQuant module usage still integrated in workflow.)

